In [1]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import seaborn as sns
import random
from scipy.ndimage import rotate, zoom
import gc
import warnings
warnings.filterwarnings('ignore')

# ---------------------------------------
# CUDA ve CUDNN Ayarları
# ---------------------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True  # Reproducibility için
print(f"Using device: {DEVICE}")

# ---------------------------------------
# Eğitim Parametreleri
# ---------------------------------------
num_epochs = 25
batch_size = 8
learning_rate = 1e-4
weight_decay = 1e-4
accumulation_steps = 4
patience = 15
patch_size = (96, 96, 96)

# Dinamik hiperparametre ayarlama fonksiyonu
def adjust_hyperparameters(fold_results):
    if not fold_results:
        return {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }
    
    try:
        last_fold = fold_results[-1]
        val_acc = last_fold['best_val_acc']
        train_loss = last_fold.get('train_losses', [])[-1] if last_fold.get('train_losses') else None
        
        new_params = {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }
        
        # Validation accuracy'ye göre learning rate ayarlama
        if val_acc < 0.6:  # Düşük accuracy
            new_params['learning_rate'] *= 1.5  # Learning rate'i artır
        elif val_acc > 0.85:  # Yüksek accuracy
            new_params['learning_rate'] *= 0.8  # Learning rate'i azalt
        
        # Overfitting kontrolü
        if train_loss and last_fold.get('val_losses'):
            train_loss = last_fold['train_losses'][-1]
            val_loss = last_fold['val_losses'][-1]
            if val_loss > train_loss * 1.2:  # Overfitting varsa
                new_params['weight_decay'] *= 1.5  # Regularization'ı artır
                new_params['batch_size'] = min(batch_size * 2, 32)  # Batch size'ı artır
        
        # Batch size sınırlamaları
        new_params['batch_size'] = max(4, min(new_params['batch_size'], 32))
        
        print("\nHiperparametre Ayarlamaları:")
        print(f"Önceki Learning Rate: {learning_rate:.2e} -> Yeni: {new_params['learning_rate']:.2e}")
        print(f"Önceki Batch Size: {batch_size} -> Yeni: {new_params['batch_size']}")
        print(f"Önceki Weight Decay: {weight_decay:.2e} -> Yeni: {new_params['weight_decay']:.2e}")
        
        return new_params
    except Exception as e:
        print(f"Hiperparametre ayarlama hatası: {str(e)}")
        return {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }

# ---------------------------------------
# Mixed Precision Training Setup
# ---------------------------------------
scaler = torch.cuda.amp.GradScaler(enabled=True)

# ---------------------------------------
# Data Augmentation Functions
# ---------------------------------------
def random_rotation(img, max_angle=15):
    try:
        angle = random.uniform(-max_angle, max_angle)
        return rotate(img, angle, axes=(0, 1), reshape=False)
    except Exception as e:
        print(f"Rotation hatası: {str(e)}")
        return img

def random_zoom(img, min_zoom=0.9, max_zoom=1.1):
    try:
        zoom_factor = random.uniform(min_zoom, max_zoom)
        return zoom(img, zoom_factor, order=1)
    except Exception as e:
        print(f"Zoom hatası: {str(e)}")
        return img

def random_flip(img, prob=0.5):
    try:
        if random.random() < prob:
            return np.flip(img, axis=0)
        return img
    except Exception as e:
        print(f"Flip hatası: {str(e)}")
        return img

def random_noise(img, mean=0, std=0.01):
    try:
        noise = np.random.normal(mean, std, img.shape)
        return img + noise
    except Exception as e:
        print(f"Noise hatası: {str(e)}")
        return img

# ---------------------------------------
# Dataset ve Model Tanımları
# ---------------------------------------
class ADNI_Dataset(Dataset):
    def __init__(self, paths, labels, transform=None, is_training=True):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.is_training = is_training

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        try:
            img = nib.load(self.paths[idx]).get_fdata()
            
            # Normalize
            img = (img - img.mean()) / (img.std() + 1e-6)
            
            # Extract patch
            center = np.array(img.shape) // 2
            patch = extract_patch(img, center, patch_size)
            
            # Apply augmentations during training
            if self.is_training:
                patch = random_rotation(patch)
                patch = random_zoom(patch)
                patch = random_flip(patch)
                patch = random_noise(patch)
            
            # Ensure patch is the correct size
            if patch.shape != patch_size:
                patch = zoom(patch, np.array(patch_size) / np.array(patch.shape), order=1)
            
            img_tensor = torch.from_numpy(patch).float().unsqueeze(0)
            
            if self.transform:
                img_tensor = self.transform(img_tensor)
            return img_tensor, self.labels[idx]
        except Exception as e:
            print(f"Veri yükleme hatası (idx={idx}): {str(e)}")
            # Hata durumunda sıfır tensör döndür
            return torch.zeros((1, *patch_size)), self.labels[idx]

def extract_patch(img, center, size):
    try:
        # Ensure center is within bounds
        center = np.clip(center, np.array(size) // 2, np.array(img.shape) - np.array(size) // 2)
        
        # Calculate start and end points
        start = center - np.array(size) // 2
        end = start + np.array(size)
        
        # Ensure start and end points are within bounds
        start = np.clip(start, 0, np.array(img.shape) - np.array(size))
        end = start + np.array(size)
        
        # Extract patch
        patch = img[start[0]:end[0], start[1]:end[1], start[2]:end[2]]
        
        # Verify patch size
        if patch.shape != size:
            print(f"Warning: Patch shape mismatch. Expected {size}, got {patch.shape}")
            # Resize patch to expected size if needed
            patch = zoom(patch, np.array(size) / np.array(patch.shape), order=1)
        
        return patch
    except Exception as e:
        print(f"Patch çıkarma hatası: {str(e)}")
        return np.zeros(size)

class ImprovedCNN3D(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        
        # Initial convolution with larger kernel
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=5, padding=2),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout3d(0.1)
        )
        
        # Residual blocks
        self.res1 = ResidualBlock(64, 128)
        self.res2 = ResidualBlock(128, 256)
        self.res3 = ResidualBlock(256, 512)
        
        # Attention mechanism
        self.attention = SelfAttention(512)
        
        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        
        # Classifier with more layers and dropout
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res1(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res2(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res3(x)
        x = F.max_pool3d(x, 2)
        
        x = self.attention(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm3d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.2)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = F.leaky_relu(out, 0.2)
        return out

class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.query = nn.Conv3d(in_channels, in_channels//8, 1)
        self.key = nn.Conv3d(in_channels, in_channels//8, 1)
        self.value = nn.Conv3d(in_channels, in_channels, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        batch_size, C, D, H, W = x.size()
        
        q = self.query(x).view(batch_size, -1, D*H*W)
        k = self.key(x).view(batch_size, -1, D*H*W)
        v = self.value(x).view(batch_size, -1, D*H*W)
        
        attention = F.softmax(torch.bmm(q.permute(0, 2, 1), k), dim=2)
        out = torch.bmm(v, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, D, H, W)
        
        return self.gamma * out + x

# ---------------------------------------
# Veri Yükleme ve Hazırlama
# ---------------------------------------
try:
    DATA_DIR = '/kaggle/input/adniunpreprocessed'
    classes = {'CN': 0, 'MCI': 1, 'AD': 2}

    pattern = os.path.join(DATA_DIR, '**', '*.nii*')
    all_files = glob.glob(pattern, recursive=True)
    valid_paths = []
    valid_labels = []

    for fp in all_files:
        parent = os.path.basename(os.path.dirname(fp))
        if parent not in classes:
            continue
        if os.path.getsize(fp) == 0:
            continue
        try:
            _ = nib.load(fp)
            valid_paths.append(fp)
            valid_labels.append(classes[parent])
        except Exception as e:
            print(f"Dosya yükleme hatası ({fp}): {str(e)}")
            continue

    if not valid_paths:
        raise ValueError("Geçerli veri bulunamadı!")

    # ---------------------------------------
    # K-Fold Cross Validation
    # ---------------------------------------
    n_splits = 2
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    all_paths = np.array(valid_paths)
    all_labels = np.array(valid_labels)

    fold_results = []

    for fold in range(n_splits):
        print(f"\nFold {fold+1}/{n_splits}")
        
        # İlk fold'dan sonra hiperparametreleri ayarla
        if fold > 0:
            new_params = adjust_hyperparameters(fold_results)
            learning_rate = new_params['learning_rate']
            batch_size = new_params['batch_size']
            weight_decay = new_params['weight_decay']
            accumulation_steps = new_params['accumulation_steps']
        
        # Data split with stratification
        train_idx, val_idx = list(kfold.split(all_paths, all_labels))[fold]
        paths_train = all_paths[train_idx]
        labels_train = all_labels[train_idx]
        paths_val = all_paths[val_idx]
        labels_val = all_labels[val_idx]
        
        # Print class distribution for this fold
        print("\nTrain set class distribution:")
        print(Counter(labels_train))
        print("\nValidation set class distribution:")
        print(Counter(labels_val))
        
        # Calculate class weights for this fold
        counter_train = Counter(labels_train)
        total_train = sum(counter_train.values())
        class_weights = [np.sqrt(total_train / (len(counter_train) * count)) for count in counter_train.values()]
        class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)
        print("\nClass weights for this fold:", class_weights)
        
        # Create weighted sampler for training
        sample_weights = [class_weights[label] for label in labels_train]
        train_sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
        
        # Dataset and DataLoader with weighted sampling
        train_ds = ADNI_Dataset(paths_train, labels_train)
        val_ds = ADNI_Dataset(paths_val, labels_val)
        
        train_loader = DataLoader(
            train_ds,
            batch_size=batch_size,
            sampler=train_sampler,
            num_workers=2,
            pin_memory=True
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
        
        # Model, Loss and Optimizer
        model = ImprovedCNN3D(num_classes=3).to(DEVICE)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
        # Cosine annealing with warm restarts
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=10,
            T_mult=2,
            eta_min=1e-6
        )
        
        # Use weighted loss function with label smoothing
        cls_loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)
        
        # Training
        best_val_acc = 0.0
        epochs_no_improve = 0
        train_losses, val_losses = [], []
        train_accs, val_accs = [], []
        
        for epoch in range(num_epochs):
            # Training
            model.train()
            total_loss = 0.0
            correct = 0
            total = 0
            
            for i, (imgs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")):
                try:
                    imgs = imgs.to(DEVICE, non_blocking=True)
                    labels = labels.to(DEVICE, non_blocking=True)
                    
                    optimizer.zero_grad()
                    
                    with torch.cuda.amp.autocast():
                        outputs = model(imgs)
                        loss = cls_loss_fn(outputs, labels) / accumulation_steps
                    
                    scaler.scale(loss).backward()
                    
                    if (i + 1) % accumulation_steps == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                    
                    total_loss += loss.item() * accumulation_steps
                    preds = outputs.argmax(dim=1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)
                    
                except Exception as e:
                    print(f"Training batch hatası (epoch {epoch+1}, batch {i}): {str(e)}")
                    continue
            
            train_loss = total_loss / len(train_loader)
            train_acc = correct / total
            train_losses.append(train_loss)
            train_accs.append(train_acc)
            
            # Validation
            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            
            with torch.no_grad():
                for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                    try:
                        imgs = imgs.to(DEVICE)
                        labels = labels.to(DEVICE)
                        
                        with torch.cuda.amp.autocast():
                            outputs = model(imgs)
                            loss = cls_loss_fn(outputs, labels)
                        
                        val_loss += loss.item()
                        preds = outputs.argmax(dim=1)
                        correct += (preds == labels).sum().item()
                        total += labels.size(0)
                        
                    except Exception as e:
                        print(f"Validation batch hatası (epoch {epoch+1}): {str(e)}")
                        continue
            
            val_loss = val_loss / len(val_loader)
            val_acc = correct / total
            val_losses.append(val_loss)
            val_accs.append(val_acc)
            
            print(f"Epoch {epoch+1}/{num_epochs}:")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            scheduler.step()
            
            if val_acc > best_val_acc + 1e-4:
                best_val_acc = val_acc
                epochs_no_improve = 0
                try:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scheduler_state_dict': scheduler.state_dict(),
                        'best_val_acc': best_val_acc,
                        'class_weights': class_weights,
                        'fold': fold + 1
                    }, f'best_model_fold_{fold+1}.pth')
                    print("--Yeni en iyi model kaydedildi--")
                except Exception as e:
                    print(f"Model kaydetme hatası: {str(e)}")
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break
            
            # Memory cleanup
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # Test evaluation for this fold
        try:
            checkpoint = torch.load(f'best_model_fold_{fold+1}.pth')
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            
            all_targets = []
            all_preds = []
            all_probs = []
            
            with torch.no_grad():
                for imgs, labels in tqdm(val_loader, desc="Testing"):
                    try:
                        imgs = imgs.to(DEVICE)
                        labels = labels.to(DEVICE)
                        
                        with torch.cuda.amp.autocast():
                            outputs = model(imgs)
                            probs = F.softmax(outputs, dim=1)
                        
                        preds = outputs.argmax(dim=1)
                        all_targets.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())
                        all_probs.extend(probs.cpu().numpy())
                        
                    except Exception as e:
                        print(f"Test batch hatası: {str(e)}")
                        continue
            
            # Calculate metrics for this fold
            fold_acc = np.mean(np.array(all_preds) == np.array(all_targets))
            fold_report = classification_report(all_targets, all_preds, target_names=["CN", "MCI", "AD"], output_dict=True)
            
            # Save fold results
            fold_results.append({
                'fold': fold + 1,
                'accuracy': fold_acc,
                'classification_report': fold_report,
                'predictions': all_preds,
                'targets': all_targets,
                'probabilities': all_probs,
                'class_weights': class_weights,
                'best_val_acc': best_val_acc,
                'best_epoch': checkpoint['epoch'],
                'train_losses': train_losses,
                'val_losses': val_losses,
                'train_accs': train_accs,
                'val_accs': val_accs
            })
            
            # Save fold results to file
            try:
                with open(f'fold_{fold+1}_results.txt', 'w') as f:
                    f.write(f"Fold {fold+1} Results:\n")
                    f.write(f"Best Validation Accuracy: {best_val_acc:.4f}\n")
                    f.write(f"Best Epoch: {checkpoint['epoch']}\n")
                    f.write(f"Final Test Accuracy: {fold_acc:.4f}\n")
                    f.write(f"Class Weights Used: {class_weights}\n\n")
                    f.write("Classification Report:\n")
                    f.write(classification_report(all_targets, all_preds, target_names=["CN", "MCI", "AD"]))
            except Exception as e:
                print(f"Sonuç dosyası yazma hatası: {str(e)}")
            
            # Plot confusion matrix for this fold
            try:
                cm = confusion_matrix(all_targets, all_preds)
                plt.figure(figsize=(8, 6))
                sns.heatmap(cm, annot=True, fmt='d', xticklabels=["CN", "MCI", "AD"], yticklabels=["CN", "MCI", "AD"])
                plt.xlabel("Predicted")
                plt.ylabel("True")
                plt.title(f"Confusion Matrix - Fold {fold+1}")
                plt.savefig(f'confusion_matrix_fold_{fold+1}.png')
                plt.close()
            except Exception as e:
                print(f"Confusion matrix çizim hatası: {str(e)}")
                
        except Exception as e:
            print(f"Test değerlendirme hatası: {str(e)}")
            continue

    # Save all fold results to a single file
    try:
        with open('all_folds_results.txt', 'w') as f:
            f.write("All Folds Results:\n\n")
            for fold_result in fold_results:
                f.write(f"Fold {fold_result['fold']}:\n")
                f.write(f"Best Validation Accuracy: {fold_result['best_val_acc']:.4f}\n")
                f.write(f"Best Epoch: {fold_result['best_epoch']}\n")
                f.write(f"Final Test Accuracy: {fold_result['accuracy']:.4f}\n")
                f.write(f"Class Weights Used: {fold_result['class_weights']}\n")
                f.write("Classification Report:\n")
                f.write(classification_report(fold_result['targets'], fold_result['predictions'], target_names=["CN", "MCI", "AD"]))
                f.write("\n" + "="*50 + "\n\n")
    except Exception as e:
        print(f"Tüm sonuçları kaydetme hatası: {str(e)}")

except Exception as e:
    print(f"Genel hata: {str(e)}")
finally:
    # Final cleanup
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


Using device: cuda

Fold 1/2

Train set class distribution:
Counter({1: 490, 0: 374, 2: 227})

Validation set class distribution:
Counter({1: 491, 0: 374, 2: 226})

Class weights for this fold: [0.861496877967819, 1.2657234296218451, 0.9860886199974187]


Epoch 1/25 - Validation: 100%|██████████| 137/137 [09:38<00:00,  4.22s/it]


Epoch 1/25:
Train Loss: 1.0259, Train Acc: 0.5060
Val Loss: 1.1031, Val Acc: 0.4500
--Yeni en iyi model kaydedildi--


Epoch 2/25 - Validation: 100%|██████████| 137/137 [08:59<00:00,  3.94s/it]


Epoch 2/25:
Train Loss: 0.9554, Train Acc: 0.5490
Val Loss: 1.0692, Val Acc: 0.4500


...

Epoch 14/25:
Train Loss: 0.9678, Train Acc: 0.5252
Val Loss: 1.0830, Val Acc: 0.4491


Epoch 15/25 - Validation: 100%|██████████| 137/137 [09:34<00:00,  4.20s/it]


Epoch 15/25:
Train Loss: 0.9587, Train Acc: 0.5316
Val Loss: 1.1164, Val Acc: 0.4491


Epoch 16/25 - Validation: 100%|██████████| 137/137 [09:31<00:00,  4.17s/it]


Epoch 16/25:
Train Loss: 0.9736, Train Acc: 0.5170
Val Loss: 1.0537, Val Acc: 0.4491
Early stopping triggered after 16 epochs
Test değerlendirme hatası: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global i